# NN with LabelEncode Target Values
y shape is 1 column with 0-3

less successful than other

In [3]:
#Import Dependencies

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [1]:
#SOURCE: https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="Body_Performance", user='postgres', password='Challah6132', host='localhost', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)



Connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [4]:
body_df = pd.read_sql('select * from body_performance', con=conn)
body_df.head()

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [5]:
#Closing the connection
conn.close()
print("Closed connection established to: ",data)


Closed connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [6]:
#LabelEncoder on gender column
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

body_df['gender_num'] = labelencoder.fit_transform(body_df['gender'])

body_df.head(10)

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking,gender_num
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C,1
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A,1
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C,1
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B,1
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B,1
5,36,F,165.4,55.40,22.0,64.0,119.0,23.8,21.0,27.0,153.0,B,0
6,42,F,164.5,63.70,32.2,72.0,135.0,22.7,0.8,18.0,146.0,D,0
7,33,M,174.9,77.20,36.9,84.0,137.0,45.9,12.3,42.0,234.0,B,1
8,54,M,166.8,67.50,27.6,85.0,165.0,40.4,18.6,34.0,148.0,C,1
9,28,M,185.0,84.60,14.4,81.0,156.0,57.9,12.1,55.0,213.0,B,1


In [7]:
# LabelEncoder on Target values ("ranking") since we have 4 classes but keeping in one column
# See bodyperf_ml_v1.3 for OneHotEncode

body_df['ranking_num'] = labelencoder.fit_transform(body_df['ranking'])

body_df.head()


,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking,gender_num,ranking_num
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C,1,2
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A,1,0
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C,1,2
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B,1,1
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B,1,1


In [8]:
#Drop columns and merge encoded df
#Ignoring encode2 for now
body_df = body_df.drop(["gender", "ranking"], axis=1)

body_df.head()


,age,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,gender_num,ranking_num
0,27,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,1,2
1,25,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,1,0
2,31,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,1,2
3,32,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1,1
4,28,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1,1


In [9]:
#Set X, y 
y = body_df["ranking_num"].values
X = body_df.drop("ranking_num", axis=1).values

print(X.shape, y.shape)



(13393, 11) (13393,)


In [10]:
y[:10]

array([2, 0, 2, 1, 1, 1, 3, 1, 2, 1])

In [11]:
#Split train/test data and scale using Standard Scaler at this time (mean of 0).  #Should we use MinMax(0-1)
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[-0.7892366 , -0.91838934, -1.3348929 , ..., -0.26990003,
        -0.56381633, -1.3150467 ],
       [ 1.71061167, -0.50110528, -0.31377541, ..., -0.48163207,
        -1.21956778,  0.76042927],
       [-0.86276155,  0.16654921,  0.95843653, ...,  0.57702813,
         1.37821682,  0.76042927],
       ...,
       [-1.00981144, -0.23881245,  1.05887432, ..., -0.05816799,
         0.49547448,  0.76042927],
       [-1.15686134,  0.07117   , -0.21333763, ..., -0.05816799,
        -0.41248907, -1.3150467 ],
       [ 1.78413662, -1.20452697, -0.58160951, ..., -0.62278677,
        -0.4881527 ,  0.76042927]])

In [12]:
# Define the model - deep neural net...length of outputs/inputs
num_input_features = len(X_train_scaled[0])
num_output_features = 1  #len(y_train[0])
print(num_input_features)
print(num_output_features)

11
1


### Test One
- 3 hidden layers
- equal nodes (10 each)
- 200 epochs
- relu, sig, sig, soft
- binary_crossentropy

In [13]:
# Define the model - deep neural net
# Try with 2 hidden layers first but will try reduced. Look at different activation functions?
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                120       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model binary vs categorical. Categorical for multi-output  --https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 725us/step - loss: -0.5337 - accuracy: 0.2491
Epoch 2/200
314/314 [==============================] - 0s 723us/step - loss: -2.3904 - accuracy: 0.2491
Epoch 3/200
314/314 [==============================] - 0s 760us/step - loss: -3.8727 - accuracy: 0.2491
Epoch 4/200
314/314 [==============================] - 0s 719us/step - loss: -5.8690 - accuracy: 0.2491
Epoch 5/200
314/314 [==============================] - 0s 716us/step - loss: -7.8864 - accuracy: 0.2491
Epoch 6/200
314/314 [==============================] - 0s 721us/step - loss: -9.7519 - accuracy: 0.2491
Epoch 7/200
314/314 [==============================] - 0s 732us/step - loss: -11.5542 - accuracy: 0.2491
Epoch 8/200
314/314 [==============================] - 0s 707us/step - loss: -13.3144 - accuracy: 0.2491
Epoch 9/200
314/314 [==============================] - 0s 733us/step - loss: -15.0511 - accuracy: 0.2491
Epoch 10/200
314/314 [==============================] - 0s 70

314/314 [==============================] - 0s 751us/step - loss: -133.8488 - accuracy: 0.2491
Epoch 79/200
314/314 [==============================] - 0s 736us/step - loss: -135.5542 - accuracy: 0.2491
Epoch 80/200
314/314 [==============================] - 0s 740us/step - loss: -137.3114 - accuracy: 0.2491
Epoch 81/200
314/314 [==============================] - 0s 732us/step - loss: -138.9897 - accuracy: 0.2491
Epoch 82/200
314/314 [==============================] - 0s 709us/step - loss: -140.7561 - accuracy: 0.2491
Epoch 83/200
314/314 [==============================] - 0s 783us/step - loss: -142.4247 - accuracy: 0.2491
Epoch 84/200
314/314 [==============================] - 0s 742us/step - loss: -144.1590 - accuracy: 0.2491
Epoch 85/200
314/314 [==============================] - 0s 764us/step - loss: -145.8833 - accuracy: 0.2491
Epoch 86/200
314/314 [==============================] - 0s 739us/step - loss: -147.5979 - accuracy: 0.2491
Epoch 87/200
314/314 [============================

314/314 [==============================] - 0s 753us/step - loss: -264.9690 - accuracy: 0.2491
Epoch 155/200
314/314 [==============================] - 0s 730us/step - loss: -266.6442 - accuracy: 0.2491
Epoch 156/200
314/314 [==============================] - 0s 716us/step - loss: -268.3874 - accuracy: 0.2491
Epoch 157/200
314/314 [==============================] - 0s 756us/step - loss: -270.1247 - accuracy: 0.2491
Epoch 158/200
314/314 [==============================] - 0s 705us/step - loss: -271.8736 - accuracy: 0.2491
Epoch 159/200
314/314 [==============================] - 0s 711us/step - loss: -273.5804 - accuracy: 0.2491
Epoch 160/200
314/314 [==============================] - 0s 776us/step - loss: -275.3451 - accuracy: 0.2491
Epoch 161/200
314/314 [==============================] - 0s 721us/step - loss: -277.0876 - accuracy: 0.2491
Epoch 162/200
314/314 [==============================] - 0s 712us/step - loss: -278.7602 - accuracy: 0.2491
Epoch 163/200
314/314 [===================

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - loss: -3.4734e+02 - accuracy: 0.2523 - 72ms/epoch - 684us/step
Loss: -347.342041015625, Accuracy: 0.25231412053108215


### Test Two
- 3 hidden layers
- nodes 14, 10, 8
- 200 epochs
- relu, sig, sig, relu
- binary_crossentropy

In [19]:
# Define the model - deep neural net
# Try with 1 hidden layer keeping activation functions and node#
hidden2_nodes_layer1 =  14
hidden2_nodes_layer2 = 10
hidden2_nodes_layer3 = 8

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(
    units=hidden2_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer3, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn2.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 14)                168       
                                                                 
 dense_4 (Dense)             (None, 10)                150       
                                                                 
 dense_5 (Dense)             (None, 8)                 88        
                                                                 
 dense_6 (Dense)             (None, 1)                 9         
                                                                 
Total params: 415
Trainable params: 415
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 765us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 2/200
314/314 [==============================] - 0s 766us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 3/200
314/314 [==============================] - 0s 871us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 4/200
314/314 [==============================] - 0s 842us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 5/200
314/314 [==============================] - 0s 789us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 6/200
314/314 [==============================] - 0s 771us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 7/200
314/314 [==============================] - 0s 770us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 8/200
314/314 [==============================] - 0s 774us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 9/200
314/314 [==============================] - 0s 786us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 10/200
314/314 [==============================] - 0s 957us

314/314 [==============================] - 0s 775us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 80/200
314/314 [==============================] - 0s 774us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 81/200
314/314 [==============================] - 0s 772us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 82/200
314/314 [==============================] - 0s 783us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 83/200
314/314 [==============================] - 0s 811us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 84/200
314/314 [==============================] - 0s 784us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 85/200
314/314 [==============================] - 0s 787us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 86/200
314/314 [==============================] - 0s 784us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 87/200
314/314 [==============================] - 0s 773us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 88/200
314/314 [==============================] - 0s 777us/ste

314/314 [==============================] - 0s 771us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 157/200
314/314 [==============================] - 0s 815us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 158/200
314/314 [==============================] - 0s 751us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 159/200
314/314 [==============================] - 0s 811us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 160/200
314/314 [==============================] - 0s 761us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 161/200
314/314 [==============================] - 0s 746us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 162/200
314/314 [==============================] - 0s 759us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 163/200
314/314 [==============================] - 0s 745us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 164/200
314/314 [==============================] - 0s 806us/step - loss: 23.0729 - accuracy: 0.2519
Epoch 165/200
314/314 [==============================] - 0s 

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

105/105 - 0s - loss: 23.3424 - accuracy: 0.2443 - 267ms/epoch - 3ms/step
Loss: 23.34238052368164, Accuracy: 0.244252011179924


### Test Three
- 3 hidden layers
- 12, 10, 8
- 200 epochs
- relu, sig, sig, soft
- categorical_crossentropy

In [23]:
# Define the model - deep neural net
# Try with 1 hidden layer keeping activation functions and node#
hidden3_nodes_layer1 =  12
hidden3_nodes_layer2 = 10
hidden3_nodes_layer3 = 8

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(
    units=hidden3_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden3_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden3_nodes_layer3, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn3.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 12)                144       
                                                                 
 dense_8 (Dense)             (None, 10)                130       
                                                                 
 dense_9 (Dense)             (None, 8)                 88        
                                                                 
 dense_10 (Dense)            (None, 1)                 9         
                                                                 
Total params: 371
Trainable params: 371
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 759us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 2/200
314/314 [==============================] - 0s 745us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 3/200
314/314 [==============================] - 0s 753us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 4/200
314/314 [==============================] - 0s 746us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 5/200
314/314 [==============================] - 0s 741us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 6/200
314/314 [==============================] - 0s 749us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 7/200
314/314 [==============================] - 0s 758us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 8/200
314/314 [==============================] - 0s 756us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 9/200
314/314 [==============================] - 0s 773us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 10/200
314/314 [===============

Epoch 77/200
314/314 [==============================] - 0s 749us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 78/200
314/314 [==============================] - 0s 773us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 79/200
314/314 [==============================] - 0s 785us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 80/200
314/314 [==============================] - 0s 776us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 81/200
314/314 [==============================] - 0s 746us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 82/200
314/314 [==============================] - 0s 746us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 83/200
314/314 [==============================] - 0s 758us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 84/200
314/314 [==============================] - 0s 744us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 85/200
314/314 [==============================] - 0s 837us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 86/200
314/314 [======

314/314 [==============================] - 0s 739us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 153/200
314/314 [==============================] - 0s 736us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 154/200
314/314 [==============================] - 0s 751us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 155/200
314/314 [==============================] - 0s 737us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 156/200
314/314 [==============================] - 0s 730us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 157/200
314/314 [==============================] - 0s 735us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 158/200
314/314 [==============================] - 0s 739us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 159/200
314/314 [==============================] - 0s 744us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 160/200
314/314 [==============================] - 0s 749us/step - loss: 1.7832e-07 - accuracy: 0.2472
Epoch 161/200
314/314 [==========

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - loss: 1.8040e-07 - accuracy: 0.2353 - 137ms/epoch - 1ms/step
Loss: 1.8039794724700187e-07, Accuracy: 0.23529411852359772
